#### Imports

In [1]:
# Telegram Bot
import telebot
import difflib
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
from datetime import datetime

# Import provisorios
from ingressos.ibge import get_estados_cidades
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Config Driver
from config.config_driver import ConfigDriver

# Definitions
from definitions import *

#### Variaveis

In [2]:
API_TOKEN = '7135171388:AAExkHE82hR68W21q-KJCsxpJmVU9gtLNPA'
bot = telebot.TeleBot(API_TOKEN)

brazilian_states = ["Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará", "Distrito Federal", 
                    "Espírito Santo", "Goiás", "Maranhão", "Mato Grosso", "Mato Grosso do Sul", 
                    "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco", "Piauí", 
                    "Rio de Janeiro", "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", 
                    "Roraima", "Santa Catarina", "São Paulo", "Sergipe", "Tocantins"]

config_driver = ConfigDriver()
driver = config_driver.get_driver()


#### Main

In [3]:
cinemas = None

@bot.message_handler(func=lambda message: True)
def reply_message(message):
    #Apenas para cinema
    if message.text == "/start":
        handle_first_contact(message)

def handle_first_contact(message):
    markup = InlineKeyboardMarkup()
    markup.row_width = 2
    markup.add(InlineKeyboardButton("Cinema", callback_data="cinema"),
               InlineKeyboardButton("Filme", callback_data="filme"))
    bot.send_message(message.chat.id, "Olá, eu sou o bot do Ingresso.com, deseja buscar por:", reply_markup=markup)

def handle_cinema_markup(message):
    # Recebe o estado enviado
    state_msg = bot.send_message(message.chat.id, "Digite a cidade do cinema que deseja buscar")
    bot.register_next_step_handler(state_msg, process_city_name)

def process_city_name(message):
    city = message.text.strip()
    
    # Obtém todos os estados e cidades
    estados_cidades = get_estados_cidades()

    # Obtém todas as cidades disponíveis de todos os estados
    todas_cidades = [cidade for cidades in estados_cidades.values() for cidade in cidades]

    # Usa difflib para encontrar a cidade mais próxima
    closest_matches = difflib.get_close_matches(city, todas_cidades, n=1, cutoff=0.6)

    # Se achar a cidade, continua para buscar os cinemas
    if closest_matches:
        city = closest_matches[0]
        bot.send_message(message.chat.id, f"Buscando cinemas em {city}...")

        # Formata o nome da cidade para a URL
        formatted_city = city.replace(" ", "-").lower()

        linkWebDriver = f"https://www.ingresso.com/cinemas?city={formatted_city}"

        driver.get(linkWebDriver)

        # Tenta fechar o popup se ele existir
        try:
            fecharPopup = WebDriverWait(driver, 99).until(
                EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
            )
            if fecharPopup.is_displayed():
                fecharPopup.click()
        except Exception:
            print("Popup button not found, continuing without clicking.")

        cinemas = getCinemasFromCity(driver, message)
    # Se não encontrar a cidade
    else:
        bot.send_message(message.chat.id, "Nenhuma cidade correspondente encontrada. Tente novamente.")
        handle_cinema_markup(message)
    
def getCinemasFromCity(driver , message):
    
    try:
        fecharPopup = WebDriverWait(driver, 99).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup button not found, continuing without clicking.")
    
    global cinemas 
    cinemas = driver.find_elements(By.CSS_SELECTOR, "h3.text-base.leading-4")  

    # Print the names of the cinemas for debugging purposes
    for cinema in cinemas:
        print(cinema.text)

    # Criar um teclado inline
    markup = InlineKeyboardMarkup()

    # Adicionar botões ao teclado com o texto do cinema e dados de callback
    for index, cinema in enumerate(cinemas):
        markup.add(InlineKeyboardButton(cinema.text, callback_data="ERIKAOXD_" + str(index)))
    # Enviar o teclado inline para o usuário
    bot.send_message(message.chat.id, "Cinemas:", reply_markup=markup)

            
def getFilmesByCinema(driver, cinemaSelecionado, message):    
    global cinemas
    
    try:
        fecharPopup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup button not found, continuing without clicking.")
    
    # Seleciona o primeiro cinema (pode ser ajustado)
    cinema_element = cinemas[cinemaSelecionado].find_element(By.XPATH, '..')

    # Rolagem para o cinema selecionado
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cinema_element)
    
    time.sleep(2)
    # Espera até que o elemento esteja visível e clicável
    WebDriverWait(driver, 10).until(EC.visibility_of(cinema_element))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(cinema_element))

    cinema_element.click()

    # Obter a data atual
    today = datetime.today()

    # Formatar a data no formato "dd/MM"
    # data = today.strftime("%d/%m")
    
    data = "28/09"
    
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{data}')]")
    except Exception:
        print(f"Element not found for date: {data}")
        # Optionally, you can retry after a delay
        time.sleep(2)
        try:
            diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{data}')]")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        except Exception:
            print(f"Element still not found for date: {data} after retrying")
            # Handle the failure case as needed
    
    
    # Close the cookie consent if it exists
    try:
        cookie_consent = driver.find_element(By.CLASS_NAME, "CookieConsent")
        if cookie_consent.is_displayed():
            close_button = cookie_consent.find_element(By.TAG_NAME, "button")
            close_button.click()
    except Exception:
        print("Cookie consent not found, continuing without clicking.")

    # Now attempt to click the desired element
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{data}')]")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        diaEscolhidoElemento.click()
    except Exception:
        print("Element click intercepted, please check the page layout.")


    # Captura os blocos de filmes
    blocos_filmes = driver.find_elements(By.CSS_SELECTOR, '.relative.my-5')
    
    horarios_filmes = ""
    
    # Itera sobre cada bloco para extrair o nome do filme, horários e links
    for bloco in blocos_filmes:
        # Extrai o nome do filme
        nome_filme = bloco.find_element(By.CSS_SELECTOR, "h3 a").text
        
        # Extrai os elementos <a> que contêm os horários
        horarios_elements = bloco.find_elements(By.CSS_SELECTOR, "a[href*='sessionId']")
        
        # Extrai o horário e o link
        horarios = []
        for horario_element in horarios_elements:
            horario = horario_element.find_element(By.CSS_SELECTOR, "span").text  # Extrai o texto do horário
            link = horario_element.get_attribute("href")  # Extrai o link
            horarios.append((horario, link))  # Adiciona à lista de horários

        # Append the movie name to the message
        horarios_filmes += f"*Nome do Filme:* {nome_filme}\n"
        horarios_filmes += f"*Horários:*\n"
        
        # Append each horario and link to the horarios_filmes
        for horario, link in horarios:
            horarios_filmes += f"  - [{horario}]({link})\n"
        
        horarios_filmes += f"---\n"

    # Send the message
    bot.send_message(message.chat.id,horarios_filmes, parse_mode='Markdown')
        

@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    if call.data == "cinema":
        handle_cinema_markup(call.message)
    elif call.data == "filme":
        bot.send_message(call.message.chat.id, "Essa função ainda não está disponível")
    elif call.data.startswith("ERIKAOXD"):
        bot.send_message(call.message.chat.id, call.data)
        index = int(call.data.split("_")[1])
        getFilmesByCinema(driver, index, message=call.message)

#### Run Bot

In [4]:
bot.polling()

Cinemark Center Vale
Cinemark Colinas
Cinépolis Jardim Oriente
Kinoplex Diamante
Kinoplex Vale Sul
Element not found for date: 28/09
Element still not found for date: 28/09 after retrying
Element click intercepted, please check the page layout.


UnboundLocalError: cannot access local variable 'horarios_filmes' where it is not associated with a value